In [1]:
import torch
import torch.nn.functional as F
from torch.utils.data import TensorDataset, DataLoader
from torch.optim import AdamW
from gpt_model import GPT
from vocab import tokens
import torch.nn as nn
import random

In [2]:
# Lista de frases como listas de tokens de treinamento
all_tokens = [
    ["vocês", "podem",   "explorar", "a", "floresta", "durante", "o", "dia"],
    ["ele",   "vai",     "vender",   "um","livro",    "no",      "mercado","amanhã"],
    ["nós",   "estamos", "felizes",  "porque","tivemos","uma","experiência","interessante","na","universidade"]
]


# Inicializa as listas de inputs e targets
inputs, targets = [], []

# Para cada frase, acumula pares (contexto → próximo token)
for tokens in all_tokens:
    for i in range(1, len(tokens)):
        # contexto: tokens até a posição i (excluindo i)
        inputs.append(tokens[:i])
        print(tokens[:i])
        # target: token na posição i
        targets.append(tokens[i])
        print(tokens[i])

# Exemplo de saída:
# inputs  = [["o"], ["o","gato"], ["o","gato","dorme"], ..., ["nós","vai","para","escola"]]
# targets = ["gato","dorme","em", ..., "amanhã"]


['vocês']
podem
['vocês', 'podem']
explorar
['vocês', 'podem', 'explorar']
a
['vocês', 'podem', 'explorar', 'a']
floresta
['vocês', 'podem', 'explorar', 'a', 'floresta']
durante
['vocês', 'podem', 'explorar', 'a', 'floresta', 'durante']
o
['vocês', 'podem', 'explorar', 'a', 'floresta', 'durante', 'o']
dia
['ele']
vai
['ele', 'vai']
vender
['ele', 'vai', 'vender']
um
['ele', 'vai', 'vender', 'um']
livro
['ele', 'vai', 'vender', 'um', 'livro']
no
['ele', 'vai', 'vender', 'um', 'livro', 'no']
mercado
['ele', 'vai', 'vender', 'um', 'livro', 'no', 'mercado']
amanhã
['nós']
estamos
['nós', 'estamos']
felizes
['nós', 'estamos', 'felizes']
porque
['nós', 'estamos', 'felizes', 'porque']
tivemos
['nós', 'estamos', 'felizes', 'porque', 'tivemos']
uma
['nós', 'estamos', 'felizes', 'porque', 'tivemos', 'uma']
experiência
['nós', 'estamos', 'felizes', 'porque', 'tivemos', 'uma', 'experiência']
interessante
['nós', 'estamos', 'felizes', 'porque', 'tivemos', 'uma', 'experiência', 'interessante']
na
['

In [ ]:
train_data = list(zip(inputs, targets))
random.shuffle(train_data)
train_data

[(['vocês', 'podem'], 'explorar'),
 (['ele'], 'vai'),
 (['nós', 'estamos', 'felizes'], 'porque'),
 (['vocês'], 'podem'),
 (['nós', 'estamos', 'felizes', 'porque', 'tivemos'], 'uma'),
 (['ele', 'vai', 'vender', 'um', 'livro', 'no', 'mercado'], 'amanhã'),
 (['vocês', 'podem', 'explorar', 'a', 'floresta', 'durante', 'o'], 'dia'),
 (['nós',
   'estamos',
   'felizes',
   'porque',
   'tivemos',
   'uma',
   'experiência',
   'interessante',
   'na'],
  'universidade'),
 (['nós',
   'estamos',
   'felizes',
   'porque',
   'tivemos',
   'uma',
   'experiência',
   'interessante'],
  'na'),
 (['ele', 'vai', 'vender', 'um'], 'livro'),
 (['ele', 'vai', 'vender', 'um', 'livro'], 'no'),
 (['ele', 'vai', 'vender'], 'um'),
 (['vocês', 'podem', 'explorar', 'a', 'floresta', 'durante'], 'o'),
 (['vocês', 'podem', 'explorar', 'a', 'floresta'], 'durante'),
 (['ele', 'vai', 'vender', 'um', 'livro', 'no'], 'mercado'),
 (['vocês', 'podem', 'explorar', 'a'], 'floresta'),
 (['nós', 'estamos', 'felizes', 'po

In [5]:
# ── 1. modelo + otimizador ───────────────────────────────────────────────
#device      = "cuda" if torch.cuda.is_available() else "cpu"
model       = GPT()
optimizer   = AdamW(model.parameters(), lr=1e-3)
criterion   = nn.CrossEntropyLoss()

In [7]:
# ── 2. loop de treino ────────────────────────────────────────────────────
num_epochs   = 100
loss_history = []

for epoch in range(num_epochs):
    epoch_loss = 0.0

    for input_tokens, target_token in train_data:          # ↳ seu dataloader/iterável
        # (opcional) envie tudo para GPU, se estiver usando
        input_tokens = list(input_tokens)                  # garante lista mutável
        model.init_tokens(input_tokens)                    # define contexto atual

        # transforma tokens em índices para o forward
        model.tokens_idx(input_tokens)                     # cria self.indices_tensor

        # ── forward ──────────────────────────────────────
        last_hidden = model.forward()                      # (n_embd,)
        logits      = model.predict_logits(last_hidden)    # (vocab_size,)

        # ── loss ────────────────────────────────────────
        target_idx  = torch.tensor([model.tokens_vocab[target_token]],
                                   dtype=torch.long,
                                   device=logits.device)   # shape (1,)
        loss        = criterion(logits.unsqueeze(0), target_idx)

        # ── backward + update ───────────────────────────
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        epoch_loss += loss.item()

    # média da loss na época
    avg_loss = epoch_loss / len(train_data)
    loss_history.append(avg_loss)
    print(f"Época {epoch+1}/{num_epochs} — Loss: {avg_loss:.4f}")


Época 1/100 — Loss: 4.4259
Época 2/100 — Loss: 4.1897
Época 3/100 — Loss: 3.9508
Época 4/100 — Loss: 3.7095
Época 5/100 — Loss: 3.4680
Época 6/100 — Loss: 3.2295
Época 7/100 — Loss: 2.9980
Época 8/100 — Loss: 2.7767
Época 9/100 — Loss: 2.5683
Época 10/100 — Loss: 2.3740
Época 11/100 — Loss: 2.1939
Época 12/100 — Loss: 2.0273
Época 13/100 — Loss: 1.8730
Época 14/100 — Loss: 1.7300
Época 15/100 — Loss: 1.5972
Época 16/100 — Loss: 1.4739
Época 17/100 — Loss: 1.3594
Época 18/100 — Loss: 1.2531
Época 19/100 — Loss: 1.1545
Época 20/100 — Loss: 1.0633
Época 21/100 — Loss: 0.9790
Época 22/100 — Loss: 0.9012
Época 23/100 — Loss: 0.8296
Época 24/100 — Loss: 0.7637
Época 25/100 — Loss: 0.7032
Época 26/100 — Loss: 0.6478
Época 27/100 — Loss: 0.5970
Época 28/100 — Loss: 0.5507
Época 29/100 — Loss: 0.5084
Época 30/100 — Loss: 0.4699
Época 31/100 — Loss: 0.4348
Época 32/100 — Loss: 0.4028
Época 33/100 — Loss: 0.3738
Época 34/100 — Loss: 0.3473
Época 35/100 — Loss: 0.3232
Época 36/100 — Loss: 0.3013
É

In [8]:
model.init_tokens(["vocês", "podem"])

In [9]:
model.predict_next_token()

'explorar'

In [10]:
model.predict_all_sentence()

Passo 1: vocês podem explorar
Passo 2: vocês podem explorar a
Passo 3: vocês podem explorar a floresta
Passo 4: vocês podem explorar a floresta durante
Passo 5: vocês podem explorar a floresta durante o
Passo 6: vocês podem explorar a floresta durante o dia
Passo 7: vocês podem explorar a floresta durante o dia estamos
Passo 8: vocês podem explorar a floresta durante o dia estamos felizes
Passo 9: vocês podem explorar a floresta durante o dia estamos felizes um
Passo 10: vocês podem explorar a floresta durante o dia estamos felizes um livro
Passo 11: vocês podem explorar a floresta durante o dia estamos felizes um livro estamos
Passo 12: vocês podem explorar a floresta durante o dia estamos felizes um livro estamos felizes


['vocês', 'podem']

In [ ]:
loss = nn.CrossEntropyLoss()
loss(torch.tensor([[0.1, 2.9]]), torch.tensor([1]))

In [ ]:
num_epochs = 2

In [ ]:
# 1) Converte tokens → índices
inputs_idx  = [[vocab[t] for t in seq] for seq in inputs]
targets_idx = [vocab[t]          for t in targets]


X = [pad_truncate(seq, context_length, pad_id) for seq in inputs_idx]
Y = targets_idx  # cada Y[i] já é um escalar

# 3) Tensores e DataLoader
X = torch.tensor(X, dtype=torch.long)   # (N, context_length)
Y = torch.tensor(Y, dtype=torch.long)   # (N,)
ds = TensorDataset(X, Y)
loader = DataLoader(ds, batch_size=batch_size, shuffle=True)

# 4) Otimizador
optimizer = AdamW(model.parameters(), lr=lr, weight_decay=wd)

# 5) Scheduler com warmup + linear decay
def lr_lambda(current_step):
    if current_step < warmup_steps:
        return float(current_step) / float(max(1, warmup_steps))
    return max(
        0.0,
        float(total_steps - current_step) / float(max(1, total_steps - warmup_steps))
    )

scheduler = torch.optim.lr_scheduler.LambdaLR(optimizer, lr_lambda)


In [ ]:

# 6) Loop de treinamento
model.train()
for epoch in range(num_epochs):
    for step, (batch_x, batch_y) in enumerate(loader):
        # forward
        model.tokens_idx([ idx_to_token[id.item()] for id in batch_x[0] ])  # adapte se necessário
        logits = model.forward()        # retorna (n_embd,), se seu forward for assim ajuste
        # aqui, se seu forward retorna apenas último token, use batchization:
        # logits = model(batch_x)  # se tiver implementado batch no forward
        # assume logits: (batch_size, vocab_size) ou (vocab_size,) + reshape
        # loss
        loss = F.cross_entropy(logits.view(-1, model.vocab_size),
                               batch_y.view(-1),
                               ignore_index=pad_id)
        # backward
        optimizer.zero_grad()
        loss.backward()
        # gradient clipping
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
        optimizer.step()
        scheduler.step()

        #if step % 50 == 0:
        print(f"Epoch {epoch+1} | Step {step}/{len(loader)} | Loss {loss.item():.4f}")

    # opcional: validação aqui

# ao final, salve checkpoint
torch.save({
    "model": model.state_dict(),
    "optimizer": optimizer.state_dict(),
    "epoch": epoch,
    "scheduler": scheduler.state_dict(),
}, "checkpoint.pth")


# Treinamento